In [ ]:
import dask.dataframe as dd
credit_record = dd.read_csv("https://s3.amazonaws.com/sqoop.oozie.ml/credit_record.csv")
credit_record.head()

In [ ]:
application_record = dd.read_csv("https://s3.amazonaws.com/sqoop.oozie.ml/application_record.csv")
application_record.head()

In [ ]:
create_report(application_record)

In [ ]:
final = dd.merge(credit_record,application_record,on='ID')
final.head()

In [ ]:
a = credit_record.shape
a[0].compute(),a[1]

In [ ]:
a = application_record.shape
a[0].compute(),a[1]

In [ ]:
a = final.shape
a[0].compute(),a[1]

In [ ]:
import pandas as pd
credit_record = pd.read_csv("credit_record.csv")
credit_record.head()

In [ ]:
application_record = pd.read_csv("application_record.csv")
application_record.head()

In [ ]:
pt=pd.pivot_table(data=credit_record,index='ID',columns='STATUS',values='MONTHS_BALANCE',aggfunc='count',fill_value=0)
pt['total_delinquent']=pt.iloc[:,1]+pt.iloc[:,2]+pt.iloc[:,3]+pt.iloc[:,4]+pt.iloc[:,5]
pt['target']=pt['total_delinquent'].apply(lambda x:1 if x!=0 else 0)
pt.head()

In [ ]:
pt = pt[['target']]

In [ ]:
pt.shape

In [ ]:
credit_record.shape

In [ ]:
application_record.shape

In [ ]:
final = pd.merge(pt,application_record,on='ID')
final.shape

In [ ]:
final = final.drop('ID',axis=1)
final.head()

In [ ]:
import sweetviz as sv
sweet_report = sv.analyze(final)
sweet_report.show_html('sweet_report.html')

In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport
design_report = ProfileReport(final)
design_report.to_file(output_file='report.html')

In [ ]:
import dtale
import pandas as pd
d = dtale.show(final)
d.open_browser()

In [ ]:
X = final.drop('target',axis=1)
y = final['target']
from xverse.transformer import WOE
clf = WOE()
clf.fit(X, y)

In [ ]:
X = clf.transform(X)
X

In [ ]:
clf.woe_df.head(50)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size= .20,random_state=20)
clf = WOE()
clf.fit(Xtrain, ytrain)
Xtrain = clf.transform(Xtrain)
Xtest = clf.transform(Xtest)
lr = LogisticRegression()
lr.fit(Xtrain,ytrain)
print("Training Accuracy")
print(lr.score(Xtrain,ytrain))
print("Testing Accuracy")
print(lr.score(Xtest,ytest))
predicted = lr.predict(Xtest)
print(confusion_matrix(ytest,predicted))
print(classification_report(ytest,predicted))

scoresdt = cross_val_score(lr,Xtrain,ytrain,cv=10,scoring='f1')
print(scoresdt)
print("Average f1")
print(np.mean(scoresdt))

In [ ]:
print(__doc__)
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import validation_curve
param_range = np.arange(1,50)
train_scores, test_scores = validation_curve(
    DecisionTreeClassifier(), Xtrain, ytrain, param_name="max_depth",scoring='f1',param_range=param_range,n_jobs=1)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.title("Validation Curve with Decision Tree Classifier")
plt.xlabel("Depth")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)
lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size= .20,random_state=20)
clf = WOE()
clf.fit(Xtrain, ytrain)
Xtrain = clf.transform(Xtrain)
Xtest = clf.transform(Xtest)
lr = RandomForestClassifier(n_estimators=100,class_weight='balanced')
lr.fit(Xtrain,ytrain)
print("Training Accuracy")
print(lr.score(Xtrain,ytrain))
print("Testing Accuracy")
print(lr.score(Xtest,ytest))
predicted = lr.predict(Xtest)
print(confusion_matrix(ytest,predicted))
print(classification_report(ytest,predicted))

scoresdt = cross_val_score(lr,Xtrain,ytrain,cv=10,scoring='f1')
print(scoresdt)
print("Average f1")
print(np.mean(scoresdt))

In [ ]:
lr.predict(Xtest)

In [ ]:
lr.predict_proba(Xtest)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
#Model Comparison
from sklearn import model_selection
lr = LogisticRegression()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier(n_estimators=100)
gb = GradientBoostingClassifier(n_estimators=100)
models = []
models.append(('Logistic Regression',lr))
models.append(('Decision Tree Classifier',dt))
models.append(('Random Forest Classifier',rf))
models.append(('Gradient Boosting Classifier',gb))

In [ ]:
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = model_selection.KFold(n_splits=10)
	cv_results = model_selection.cross_val_score(model, Xtrain, ytrain, cv=kfold,scoring='recall')
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, np.median(cv_results), cv_results.std())
	print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()